# 한국어 데이터로 챗봇 만들기
프로젝트 제출 루브릭
| **학습 목표** | **평가 기준** |
|----------------|----------------|
| 한국어 전처리를 통해 학습 데이터셋을 구축하였다. | 공백과 특수문자 처리, 토크나이징, 병렬데이터 구축의 과정이 적절히 진행되었다. |
| 트랜스포머 모델을 구현하여 한국어 챗봇 모델 학습을 정상적으로 진행하였다. | 구현한 트랜스포머 모델이 한국어 병렬 데이터 학습 시 안정적으로 수렴하였다. |
| 한국어 입력문장에 대해 한국어로 답변하는 함수를 구현하였다. | 한국어 입력문장에 맥락에 맞는 한국어로 답변을 리턴하였다. |

# Step 0. Library

In [36]:
!pip install sentencepiece

In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import sentencepiece as spm

import math
import os
import re
import urllib.request
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Step 1. 데이터 수집하기

In [38]:
#!wget https://github.com/songys/Chatbot_data/raw/master/ChatbotData.csv
#!mv ChatbotData.csv data/chatbot

데이터 자체가 이런식으로 표현되어 있다.

| **Q** | **A** | **Label** |
|----------------|----------------|----------------|
|12시 땡!, |하루가 또 가네요., |0|

---

데이터의 Label은 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2로 레이블링로 되어있다.

--- 



# Step 2. 데이터 전처리하기

In [39]:
extract_dir = os.path.join("data/chatbot/")
data = pd.read_csv(extract_dir+"ChatbotData.csv")
#data = data[data.label == 2]
data = data.drop(columns='label')
data

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.
...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.
11820,흑기사 해주는 짝남.,설렜겠어요.
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.


In [40]:
def preprocess_sentence(sentence):
    # 괄호, 따옴표, 기타 기호 제거
    sentence = re.sub(r"[\(\)\[\]\"\'…]", "", sentence)
    
    # 구두점 앞뒤 공백 정리
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    
    # 한글, 구두점, 공백, 숫자 남기기
    sentence = re.sub(r"[^가-힣0-9?.!,\s]", "", sentence)
    
    # 다중 공백 축소
    sentence = re.sub(r"\s+", " ", sentence).strip()
    
    return sentence

In [41]:
def read_korean_chatbot_data(df):
    """
    한국어 Q-A 챗봇 데이터셋을 읽어 (질문, 답변) 쌍으로 구성하는 함수.

    Args:
        path (str): CSV 파일 경로 (예: 'chatbot_data.csv')
        max_samples (int, optional): 최대 추출할 (질문, 답변) 쌍 수. 기본값 None → 전체 사용

    Returns:
        list[tuple[str, str]]: (질문, 답변) 쌍의 리스트
    """
    # 전처리 및 (질문, 답변) 쌍 구성
    pairs = []
    for _, row in df.iterrows():
        q = preprocess_sentence(str(row['Q']))
        a = preprocess_sentence(str(row['A']))

        if q and a:
            pairs.append((q, a))


    return pairs

In [42]:
pairs = read_korean_chatbot_data(data)
print('전체 샘플 수 :', len(pairs))

전체 샘플 수 : 11790


# Step 3. SentencePiece 사용하기

In [43]:
corpus_file = extract_dir+"corpus.txt"
with open(corpus_file, 'w', encoding='utf-8') as f:
    for q, a in pairs:
        f.write(q + "\n")
        f.write(a + "\n")

In [44]:
spm.SentencePieceTrainer.Train(
    input=corpus_file,                        # 학습할 말뭉치 파일 (한 줄에 한 문장)
    model_prefix=extract_dir + "spm_korean",  # 출력 모델 파일 접두사
    vocab_size=1200,                          # 토큰 개수 
    character_coverage=0.9995,                # 한글 데이터는 0.9995 권장 (한자, 이모지 등 제외)
    model_type="unigram",                     # BPE보다 Unigram이 한국어에 자주 쓰임
    max_sentence_length=999999,               # 문장 길이 제한 (충분히 크게 설정)
    bos_id=1,                                 # 문장 시작 토큰 <s>
    eos_id=2,                                 # 문장 종료 토큰 </s>
    pad_id=0,                                 # 패딩 토큰
    unk_id=3,                                 # 알 수 없는 토큰
)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/chatbot/corpus.txt
  input_format: 
  model_prefix: data/chatbot/spm_korean
  model_type: UNIGRAM
  vocab_size: 1200
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 999999
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 3
  bos_id: 1
  eos_id: 2
  pad_id: 0
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_diff

In [45]:
sp = spm.SentencePieceProcessor()
sp.Load(extract_dir+"spm_korean.model")

True

In [46]:
# 예제 문장
sentence = "짝사랑만큼 고통스러운 건 없겠지."

sentence = preprocess_sentence(sentence)
print("전처리 후의 문장:", sentence)

# 1. 토크나이징 (subword 단위로 분할)
tokens = sp.encode(sentence, out_type=str)
print("Tokenized:", tokens)

# 2. 인코딩 (서브워드를 정수 ID로 변환)
encoded = sp.encode(sentence, out_type=int)
print("Encoded:", encoded)

# 3. 디코딩 (정수 ID → 원본 문장 복원)
decoded = sp.decode(encoded)
print("Decoded:", decoded)


전처리 후의 문장: 짝사랑만큼 고통스러운 건 없겠지 .
Tokenized: ['▁짝사랑', '만', '큼', '▁', '고', '통', '스', '러', '운', '▁건', '▁', '없', '겠지', '▁.']
Encoded: [273, 31, 1154, 4, 14, 251, 123, 116, 169, 137, 4, 1195, 156, 5]
Decoded: 짝사랑만큼 고통스러운 건 없겠지 .


# Step 4. 모델 구성하기

## Step 4-1. Model class 선언

In [47]:
class PositionalEncoding(nn.Module):
    def __init__(self, position, d_model):
        """
        Positional Encoding 클래스

        Args:
            position (int): 문장의 최대 길이 (max sequence length)
            d_model (int): 임베딩 벡터의 차원 (model dimension)

        역할:
            단어의 순서(위치) 정보를 임베딩 벡터에 더해줌으로써,
            Transformer가 RNN처럼 순서 정보를 학습하지 않아도
            위치 정보를 인코딩할 수 있도록 해줌.
        """
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        self.position = position

        # 위치별로 미리 계산된 사인/코사인 기반 위치 인코딩 행렬 생성
        self.pos_encoding = self._build_pos_encoding(position, d_model)

    def _get_angles(self, position, i, d_model):
        """
        각 위치(pos)와 차원(i)에 대해 angle 값을 계산.

        Args:
            position (Tensor): [position, 1] 위치 인덱스
            i (Tensor): [1, d_model] 차원 인덱스
            d_model (int): 임베딩 차원

        Returns:
            Tensor: 위치-차원별 angle 값
        """
        # (2 * (i // 2)) : 짝수/홀수 차원 구분용
        return 1.0 / (10000.0 ** ((2.0 * (i // 2)) / d_model)) * position

        # 사실 위처럼 지수계산보다는 log계산으로 푸는 것이 더 안정적이라 한다. 
        # angle_rates = torch.exp(- (2 * (i // 2)) * torch.log(torch.tensor(10000.0)) / d_model)
        # return position * angle_rates.unsqueeze(0)

    def _build_pos_encoding(self, position, d_model):
        """
        전체 위치 인코딩 행렬을 생성.

        Returns:
            pos_encoding: [1, position, d_model]
        """
        # 각 위치에 대해 0 ~ position-1까지의 인덱스 생성 → shape: [position, 1]
        pos = torch.arange(position, dtype=torch.float32).unsqueeze(1)
        # 각 차원에 대해 0 ~ d_model-1까지의 인덱스 생성 → shape: [1, d_model]
        i = torch.arange(d_model, dtype=torch.float32).unsqueeze(0)

        # 각 위치와 차원에 대한 각도(angle) 계산
        angle_rads = self._get_angles(pos, i, d_model)

        # 짝수 인덱스(0, 2, 4, ...)에는 sin, 홀수 인덱스(1, 3, 5, ...)에는 cos 적용
        sines = torch.sin(angle_rads[:, 0::2])
        cosines = torch.cos(angle_rads[:, 1::2])

        # 최종 위치 인코딩 행렬 초기화
        pos_encoding = torch.zeros(position, d_model)
        pos_encoding[:, 0::2] = sines  # 짝수 차원 → sin 값
        pos_encoding[:, 1::2] = cosines  # 홀수 차원 → cos 값

        # 배치 차원 추가: shape [1, position, d_model]
        pos_encoding = pos_encoding.unsqueeze(0)
        return pos_encoding

    def forward(self, x):
        """
        입력 임베딩(x)에 위치 인코딩을 더함.
        Args:
            x (Tensor): [batch_size, seq_len, d_model]
        Returns:
            Tensor: [batch_size, seq_len, d_model]
        """
        # 입력 길이(seq_len)에 맞는 부분만 잘라서 더함
        return x + self.pos_encoding[:, :x.size(1), :].to(x.device)


In [48]:
def scaled_dot_product_attention(query, key, value, mask=None):
    """
    Scaled Dot-Product Attention
    -------------------------------------------------------
    query: (batch_size, heads, seq_len_q, depth)
    key:   (batch_size, heads, seq_len_k, depth)
    value: (batch_size, heads, seq_len_v, depth)
    mask:  (optional) attention mask tensor
    return: (output, attention_weights)
    -------------------------------------------------------
    핵심 수식:
        Attention(Q, K, V) = softmax( (QK^T) / sqrt(d_k) ) * V
    """

    # 1) Query와 Key의 내적(dot-product)을 통해 유사도(score) 계산
    #    - key.transpose(-1, -2): 마지막 두 차원을 전치하여 (depth, seq_len_k)
    #    - 결과 shape: (batch_size, heads, seq_len_q, seq_len_k)
    matmul_qk = torch.matmul(query, key.transpose(-1, -2))

    # 2) Key 벡터의 차원(depth)에 따라 스케일링 (정규화)
    #    - 큰 값으로 인한 softmax gradient vanishing 방지
    #    - sqrt(depth)로 나눠줌
    depth = key.size(-1)
    logits = matmul_qk / math.sqrt(depth)

    # 3) 마스크(mask)가 주어진 경우 적용
    #    - 패딩 토큰 또는 미래 토큰(Decoder의 causal mask) 무시용
    #    - 매우 작은 값(-1e9)을 더해 softmax에서 해당 위치의 확률을 0으로 만듦
    if mask is not None:
        logits = logits + (mask * -1e9)

    # 4) Softmax를 통해 attention weight 계산
    #    - 각 query에 대해 모든 key의 가중치 분포 생성
    #    - dim=-1: seq_len_k 차원(즉, key 차원)에 대해 정규화
    attention_weights = F.softmax(logits, dim=-1)

    # 5) attention weight를 value에 곱해 weighted sum 계산
    #    - 결과: (batch_size, heads, seq_len_q, depth)
    output = torch.matmul(attention_weights, value)

    # output: context vector, attention_weights: 각 token 간 주의 분포
    return output, attention_weights


In [49]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads  # 병렬적으로 나눠 계산할 헤드(head)의 개수
        self.d_model = d_model      # 입력/출력의 전체 차원 (embedding dimension)

        # d_model은 num_heads로 정확히 나누어떨어져야 함
        # (예: d_model=512, num_heads=8 → head당 depth=64)
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        # 각 head에서 사용할 벡터의 차원
        self.depth = d_model // num_heads

        # Q, K, V를 생성하기 위한 선형 변환 (각각 d_model → d_model)
        # 학습 가능한 가중치 행렬을 통해 입력을 head 차원으로 투영
        self.query_dense = nn.Linear(d_model, d_model)
        self.key_dense = nn.Linear(d_model, d_model)
        self.value_dense = nn.Linear(d_model, d_model)

        # 모든 head를 다시 결합한 후, 최종 출력 차원 복원용 선형 변환
        self.out_dense = nn.Linear(d_model, d_model)

    def split_heads(self, x, batch_size):
        """
        입력 x를 num_heads 개로 분리하는 함수
        ---------------------------------------------------
        x: (batch_size, seq_len, d_model)
        반환: (batch_size, num_heads, seq_len, depth)
        ---------------------------------------------------
        - d_model을 num_heads로 나누어 각 head가 처리할 부분 벡터로 분할
        - 이후 permute를 통해 head 차원을 앞으로 이동
        """
        # (batch_size, seq_len, d_model) → (batch_size, seq_len, num_heads, depth)
        x = x.view(batch_size, -1, self.num_heads, self.depth)

        # (batch_size, num_heads, seq_len, depth) 형태로 차원 재배치
        x = x.permute(0, 2, 1, 3)
        return x

    def forward(self, query, key, value, mask=None):
        """
        Multi-Head Attention의 순전파(forward) 과정
        ---------------------------------------------------
        query, key, value: (batch_size, seq_len, d_model)
        mask: (optional) Attention mask
        ---------------------------------------------------
        출력: (batch_size, seq_len, d_model)
        """
        batch_size = query.size(0)

        # 1) 입력 벡터에 각각 Linear layer 적용 → Q, K, V 생성
        #    shape: (batch_size, seq_len, d_model)
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # 2) Head별로 분리 (num_heads, depth 구조로 변환)
        #    shape: (batch_size, num_heads, seq_len, depth)
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # 3) 각 head별로 Scaled Dot-Product Attention 수행
        #    반환값: (batch_size, num_heads, seq_len, depth)
        scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)

        # 4) 모든 head의 결과를 다시 결합하기 위해 차원 순서 재정렬
        #    (batch_size, num_heads, seq_len, depth)
        #      → (batch_size, seq_len, num_heads, depth)
        scaled_attention = scaled_attention.permute(0, 2, 1, 3).contiguous()

        # 5) num_heads와 depth를 결합해 원래 d_model 차원으로 복원
        #    shape: (batch_size, seq_len, d_model)
        concat_attention = scaled_attention.view(batch_size, -1, self.d_model)

        # 6) 최종 선형 변환을 통해 head 통합 결과를 출력 차원으로 투영
        #    (batch_size, seq_len, d_model)
        output = self.out_dense(concat_attention)

        return output

In [50]:
def create_padding_mask(x):
    # x == 0 위치를 찾아 float형 1로 변환
    mask = (x == 0).float()
    # (batch_size, seq_len) -> (batch_size, 1, 1, seq_len)
    mask = mask.unsqueeze(1).unsqueeze(2)
    return mask

In [51]:
def create_look_ahead_mask(x):
    seq_len = x.size(1)

    # 1) Look-ahead 마스크 생성
    # torch.ones((seq_len, seq_len)) → 1로 채워진 정방행렬 생성
    # torch.tril() → 하삼각(자기 자신 포함) 부분만 남기고 나머지를 0으로 만듦
    # 1 - tril(...) → 상삼각(즉, 미래 토큰 위치)이 1, 나머지는 0
    # => Decoder가 아직 보지 않은 미래 단어를 참고하지 않도록 차단
    look_ahead_mask = 1 - torch.tril(torch.ones((seq_len, seq_len)))

    # 2) 입력 x에서 패딩 위치(0인 부분)를 찾아 패딩 마스크 생성
    # shape: (batch_size, 1, 1, seq_len)
    padding_mask = create_padding_mask(x)

    # 3) Look-ahead 마스크 차원 확장
    # (seq_len, seq_len) → (1, seq_len, seq_len) → (1, 1, seq_len, seq_len)
    # => Attention 연산 시 브로드캐스팅이 가능하도록 형태 맞춤
    look_ahead_mask = look_ahead_mask.unsqueeze(0).unsqueeze(1)
    look_ahead_mask = look_ahead_mask.to(x.device)

    # 4) Look-ahead 마스크와 패딩 마스크를 결합
    # torch.max()를 사용하여 둘 중 하나라도 1인 위치는 모두 마스킹 처리
    # 최종 shape: (batch_size, 1, seq_len, seq_len)
    combined_mask = torch.max(look_ahead_mask, padding_mask)
    
    return combined_mask


In [52]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, ff_dim, dropout=0.1):
        super(EncoderLayer, self).__init__()

        # (1) 멀티-헤드 어텐션 (Self-Attention)
        # 입력 문장 내 단어들이 서로 어떤 관계를 맺고 있는지 학습
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.dropout1 = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(d_model, eps=1e-6)  # 잔차 연결 후 정규화

        # (2) 포지션별 피드포워드 네트워크 (Feed Forward Network)
        # 각 위치의 특징을 비선형 변환을 통해 확장
        self.ffn = nn.Sequential(
            nn.Linear(d_model, ff_dim),  # 차원 확장
            nn.ReLU(),                   # 비선형 활성화
            nn.Linear(ff_dim, d_model)   # 원래 차원으로 복귀
        )
        self.dropout2 = nn.Dropout(dropout)
        self.norm2 = nn.LayerNorm(d_model, eps=1e-6)  # 두 번째 정규화

    def forward(self, x, mask=None):
        """
        Args:
            x: 입력 텐서 (batch_size, seq_len, d_model)
            mask: 패딩 마스크 (선택적)

        Returns:
            out2: 인코더 레이어의 출력 (batch_size, seq_len, d_model)
        """

        # (1) 멀티-헤드 셀프 어텐션
        # Query, Key, Value 모두 같은 입력 x를 사용
        attn_output = self.mha(x, x, x, mask)         # (batch_size, seq_len, d_model)
        attn_output = self.dropout1(attn_output)      # 드롭아웃으로 과적합 방지
        out1 = self.norm1(x + attn_output)            # 잔차 연결(residual) + LayerNorm

        # (2) 포지션별 피드포워드 신경망
        ffn_output = self.ffn(out1)                   # (batch_size, seq_len, d_model)
        ffn_output = self.dropout2(ffn_output)        # 드롭아웃 적용
        out2 = self.norm2(out1 + ffn_output)          # 잔차 연결 + LayerNorm

        return out2


In [53]:
class Encoder(nn.Module):
    def __init__(self,
                 vocab_size,
                 num_layers,
                 ff_dim,
                 d_model,
                 num_heads,
                 dropout=0.1):
        super(Encoder, self).__init__()
        self.d_model = d_model

        # (1) 단어 임베딩 (Word Embedding)
        # 입력 토큰 ID를 고정 크기 벡터(d_model 차원)로 변환
        self.embedding = nn.Embedding(vocab_size, d_model)

        # (2) 위치 임베딩 (Positional Encoding)
        # 문장 내 단어 순서(위치) 정보를 추가해 순서 의존성 학습 가능하게 함
        self.pos_encoding = PositionalEncoding(position=vocab_size, d_model=d_model)

        # 드롭아웃: 학습 시 일부 뉴런 비활성화로 과적합 방지
        self.dropout = nn.Dropout(dropout)

        # (3) 인코더 블록(EncoderLayer)들을 num_layers 개만큼 쌓기
        self.enc_layers = nn.ModuleList([
            EncoderLayer(d_model, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])

    def forward(self, x, mask=None):
        """
        Args:
            x: 입력 토큰 시퀀스 (batch_size, seq_len)
            mask: 패딩 마스크 (선택적)

        Returns:
            x: 인코더의 출력 벡터 (batch_size, seq_len, d_model)
        """

        # (1) 단어 임베딩 + 스케일링
        # sqrt(d_model)로 스케일링해 학습 안정화 (Attention 계산 시 값이 너무 작아지는 것 방지)
        x = self.embedding(x) * math.sqrt(self.d_model)

        # (2) 포지셔널 인코딩 추가 + 드롭아웃
        # 위치 정보가 추가된 임베딩이 self-attention으로 입력됨
        x = self.pos_encoding(x)                     # (batch_size, seq_len, d_model)
        x = self.dropout(x)

        # (3) N개의 EncoderLayer를 순차적으로 통과
        # 각 레이어에서 self-attention → feed-forward → 정규화 과정을 거침
        for layer in self.enc_layers:
            x = layer(x, mask)

        # (4) 최종 인코더 출력
        return x


In [54]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, ff_dim, dropout=0.1):
        super(DecoderLayer, self).__init__()

        # (1) 디코더 내부의 셀프 어텐션 (Masked Multi-Head Attention)
        # => 이전 시점까지만 참조하도록 Look-ahead Mask 사용
        self.self_mha = MultiHeadAttention(d_model, num_heads)
        self.norm1 = nn.LayerNorm(d_model, eps=1e-6)

        # (2) 인코더-디코더 어텐션
        # => 디코더가 인코더의 출력(컨텍스트)을 참고하도록 하는 Cross-Attention
        self.encdec_mha = MultiHeadAttention(d_model, num_heads)
        self.norm2 = nn.LayerNorm(d_model, eps=1e-6)

        # (3) 포지션별 피드포워드 네트워크 (Position-wise Feed Forward Network)
        # => 각 위치의 피처를 독립적으로 비선형 변환
        self.ffn = nn.Sequential(
            nn.Linear(d_model, ff_dim),  # 확장
            nn.ReLU(),                   # 비선형 활성화
            nn.Linear(ff_dim, d_model)   # 축소 (원래 차원 복원)
        )
        self.norm3 = nn.LayerNorm(d_model, eps=1e-6)

        # 드롭아웃: 각 서브 레이어의 출력에 적용
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

    def forward(self, x, enc_outputs, look_ahead_mask=None, padding_mask=None):
        """
        Args:
            x: 디코더 입력 (batch_size, target_seq_len, d_model)
            enc_outputs: 인코더 출력 (batch_size, input_seq_len, d_model)
            look_ahead_mask: 미래 단어를 가리는 마스크
            padding_mask: 인코더 출력에서 패딩 위치를 가리는 마스크

        Returns:
            out3: 디코더 레이어의 출력 (batch_size, target_seq_len, d_model)
        """

        # (1) Masked Self-Attention
        # => 디코더가 다음 단어를 미리 보지 않도록 Look-ahead Mask 사용
        self_attn_out = self.self_mha(x, x, x, mask=look_ahead_mask)
        self_attn_out = self.dropout1(self_attn_out)
        out1 = self.norm1(x + self_attn_out)  # 잔차 연결 + LayerNorm

        # (2) Encoder-Decoder Attention (Cross-Attention)
        # => 인코더 출력(enc_outputs)을 Key, Value로 사용해 입력 문맥 정보 활용
        encdec_attn_out = self.encdec_mha(out1, enc_outputs, enc_outputs, mask=padding_mask)
        encdec_attn_out = self.dropout2(encdec_attn_out)
        out2 = self.norm2(out1 + encdec_attn_out)  # 잔차 연결 + LayerNorm

        # (3) Position-wise Feed Forward Network
        # => 각 시점별로 독립적인 비선형 변환 수행
        ffn_out = self.ffn(out2)
        ffn_out = self.dropout3(ffn_out)
        out3 = self.norm3(out2 + ffn_out)  # 잔차 연결 + LayerNorm

        return out3


In [55]:
class Decoder(nn.Module):
    def __init__(self,
                 vocab_size,
                 num_layers,
                 ff_dim,
                 d_model,
                 num_heads,
                 dropout=0.1):
        super(Decoder, self).__init__()
        self.d_model = d_model

        # (1) 토큰 임베딩 레이어
        # 입력된 토큰 인덱스(정수 시퀀스)를 d_model 차원의 임베딩 벡터로 변환
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=sp.pad_id())

        # (2) 포지셔널 인코딩 (Positional Encoding)
        # 디코더 입력 시퀀스의 위치 정보를 벡터에 더해줌
        # ※ 실제 구현에서는 vocab_size 대신 max_seq_len을 사용하는 경우가 많음
        self.pos_encoding = PositionalEncoding(position=vocab_size, d_model=d_model)

        # (3) 드롭아웃
        self.dropout = nn.Dropout(dropout)

        # (4) DecoderLayer를 num_layers만큼 스택
        # 각 레이어는 (Self-Attention → Encoder-Decoder Attention → FFN) 순서로 구성
        self.dec_layers = nn.ModuleList([
            DecoderLayer(d_model, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])

    def forward(self, x, enc_outputs, look_ahead_mask=None, padding_mask=None):
        """
        Args:
            x: 디코더 입력 (batch_size, target_seq_len)
            enc_outputs: 인코더 출력 (batch_size, input_seq_len, d_model)
            look_ahead_mask: 미래 단어 마스크 (디코더 셀프 어텐션용)
            padding_mask: 인코더 출력의 패딩 위치 마스크 (Cross-Attention용)
        Returns:
            x: 디코더 출력 (batch_size, target_seq_len, d_model)
        """

        # (1) 임베딩 + 스케일링
        # 임베딩 결과를 √d_model로 스케일링해 학습 안정화
        x = self.embedding(x) * math.sqrt(self.d_model)

        # (2) 포지셔널 인코딩 추가 + 드롭아웃
        # 위치 정보(positional encoding)를 더해 순서 정보 보존
        x = self.pos_encoding(x)    # (batch_size, tgt_seq_len, d_model)
        x = self.dropout(x)

        # (3) 여러 개의 DecoderLayer를 순차적으로 통과
        # 각 layer는 Masked Self-Attention → Cross-Attention → FFN 구조
        for layer in self.dec_layers:
            x = layer(x, enc_outputs, look_ahead_mask, padding_mask)

        # (4) 디코더의 최종 출력 반환
        # 각 타임스텝별로 문맥이 반영된 벡터 (d_model 차원)
        return x

In [56]:
class Transformer(nn.Module):
    def __init__(self,
                 vocab_size,
                 num_layers,      # 인코더/디코더 층 수
                 units,           # Feed-Forward Network의 은닉 차원 (ff_dim)
                 d_model,         # 임베딩 및 내부 표현 차원
                 num_heads,       # 멀티헤드 어텐션 헤드 수
                 dropout=0.1):
        super(Transformer, self).__init__()

        # 1) 인코더 초기화
        self.encoder = Encoder(
            vocab_size=vocab_size,
            num_layers=num_layers,
            ff_dim=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout
        )

        # 2) 디코더 초기화
        self.decoder = Decoder(
            vocab_size=vocab_size,
            num_layers=num_layers,
            ff_dim=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout
        )

        # 3) 최종 출력층: d_model -> vocab_size
        # 디코더 출력 벡터를 단어 확률로 변환
        self.final_linear = nn.Linear(d_model, vocab_size)

    def forward(self, inputs, dec_inputs):
        """
        inputs: 인코더 입력 (batch_size, src_seq_len)
        dec_inputs: 디코더 입력 (batch_size, tgt_seq_len)
        """
        # 1) 인코더 패딩 마스크 생성
        # PAD 토큰 위치는 1, 실제 토큰은 0
        enc_padding_mask = create_padding_mask(inputs)     # shape: (batch_size, 1, 1, src_seq_len)

        # 2) 디코더 Look-Ahead Mask + 패딩 마스크
        # 디코더에서 미래 토큰을 보지 않도록 상삼각 마스크 적용
        look_ahead_mask = create_look_ahead_mask(dec_inputs)  # shape: (batch_size, 1, tgt_seq_len, tgt_seq_len)

        # 3) 디코더에서 인코더 출력 쪽을 마스킹할 때 사용
        # 인코더 입력의 PAD 토큰 위치를 마스크
        dec_padding_mask = create_padding_mask(inputs)        # shape: (batch_size, 1, 1, src_seq_len)

        # 4) 인코더 수행
        enc_outputs = self.encoder(
            x=inputs,
            mask=enc_padding_mask
        )  # shape: (batch_size, src_seq_len, d_model)

        # 5) 디코더 수행
        dec_outputs = self.decoder(
            x=dec_inputs,           # 디코더 입력 (batch_size, tgt_seq_len)
            enc_outputs=enc_outputs,# 인코더 출력 (batch_size, src_seq_len, d_model)
            look_ahead_mask=look_ahead_mask,
            padding_mask=dec_padding_mask
        )  # shape: (batch_size, tgt_seq_len, d_model)

        # 6) 최종 Linear 레이어: d_model -> vocab_size
        # 각 위치마다 단어 확률(logits) 출력
        logits = self.final_linear(dec_outputs)  # shape: (batch_size, tgt_seq_len, vocab_size)

        return logits


## Step 4-2. Dataset & DataLoader

In [57]:
class KoreanChatDataset(Dataset):
    def __init__(self, pairs, sp, max_length=40):
        """
        한국어 챗봇 병렬 데이터셋 클래스

        Args:
            pairs (list of tuple): (질문, 답변) 쌍의 리스트
            sp (SentencePieceProcessor): 학습된 SentencePiece 토크나이저
            max_length (int): 최대 문장 길이 (패딩 포함)
        """
        super().__init__()
        self.sp = sp
        self.max_length = max_length
        self.data = []

        for q_text, a_text in pairs:
            # 전처리: 불필요한 공백 및 특수문자 제거
            q_text = q_text.strip()
            a_text = a_text.strip()
            if not q_text or not a_text:
                continue

            # SentencePiece 토큰화 (문자열 → ID 시퀀스)
            q_ids = sp.encode(q_text, out_type=int)
            a_ids = sp.encode(a_text, out_type=int)

            # 시작/종료 토큰 추가 (없을 경우 기본값 지정)
            bos_id = sp.bos_id() if sp.bos_id() >= 0 else 1
            eos_id = sp.eos_id() if sp.eos_id() >= 0 else 2
            pad_id = sp.pad_id() if sp.pad_id() >= 0 else 0

            q_tokens = [bos_id] + q_ids + [eos_id]
            a_tokens = [bos_id] + a_ids + [eos_id]

            # 최대 길이 초과 문장 제외
            if len(q_tokens) > max_length or len(a_tokens) > max_length:
                continue

            # 패딩 추가 (max_length에 맞춰 0으로 채움)
            q_tokens += [pad_id] * (max_length - len(q_tokens))
            a_tokens += [pad_id] * (max_length - len(a_tokens))

            # 디코더 입력(dec_input)과 타깃(target) 분리 (Teacher Forcing)
            dec_input = a_tokens[:-1]  # <BOS>로 시작, 마지막 토큰 제외
            target = a_tokens[1:]      # 첫 토큰 제외, <EOS>까지 포함

            # 데이터 저장
            self.data.append({
                "enc_input": q_tokens,  # Encoder 입력
                "dec_input": dec_input,  # Decoder 입력
                "target": target         # Decoder 타깃
            })

    def __len__(self):
        """전체 샘플 수 반환"""
        return len(self.data)

    def __getitem__(self, idx):
        """인덱스에 해당하는 샘플 반환"""
        sample = self.data[idx]
        enc_input = torch.tensor(sample["enc_input"], dtype=torch.long)
        dec_input = torch.tensor(sample["dec_input"], dtype=torch.long)
        target = torch.tensor(sample["target"], dtype=torch.long)
        return enc_input, dec_input, target


In [58]:
def sequence_len(data):
    """
    문장길이의 평균값, 최대값, 표준편차를 계산해 본다.
    """
    num_tokens = [len(tokens) for tokens in data]
    num_tokens = np.array(num_tokens)

    sequence_mean = np.mean(num_tokens)
    sequence_max = np.max(num_tokens)
    sequence_std = np.std(num_tokens)
    
    print('문장길이 평균 : ', sequence_mean)
    print('문장길이 최대 : ', sequence_max)
    print('문장길이 표준편차 : ', sequence_std)

    max_tokens = sequence_mean + 2 * sequence_std
    maxlen = int(max_tokens)
    print(f'적정 문장 길이 : {maxlen}')
    print(f'전체 문장의 {np.sum([len(tokens) for tokens in data] < max_tokens) / len(num_tokens):.2f}%가 maxlen 설정값 이내에 포함됩니다. ')
    return sequence_mean, sequence_max, sequence_std, maxlen
    
sequence_len(data.Q)

문장길이 평균 :  12.893299406276505
문장길이 최대 :  56
문장길이 표준편차 :  6.162788620730713
적정 문장 길이 : 25
전체 문장의 0.96%가 maxlen 설정값 이내에 포함됩니다. 


(np.float64(12.893299406276505),
 np.int64(56),
 np.float64(6.162788620730713),
 25)

In [59]:
sequence_len(data.A)

문장길이 평균 :  15.050296861747244
문장길이 최대 :  76
문장길이 표준편차 :  6.6777209494680045
적정 문장 길이 : 28
전체 문장의 0.96%가 maxlen 설정값 이내에 포함됩니다. 


(np.float64(15.050296861747244),
 np.int64(76),
 np.float64(6.6777209494680045),
 28)

In [60]:
dataset = KoreanChatDataset(pairs, sp, max_length=28)

In [61]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [62]:
for encoder_input, decoder_input, decoder_label in dataloader:
    print(encoder_input.size())
    print(decoder_input.size())
    print(decoder_label.size())
    break

torch.Size([32, 28])
torch.Size([32, 27])
torch.Size([32, 27])


## Step 4-3. Model Train

In [63]:
# 예시: 하이퍼파라미터 설정
NUM_LAYERS = 6      # 인코더/디코더 층 수
D_MODEL = 256       # 임베딩 및 내부 표현 차원
NUM_HEADS = 8       # 멀티헤드 어텐션 헤드 수
UNITS = 512         # Feed-Forward Network 은닉 차원 (ff_dim)
DROPOUT = 0.1       # 드롭아웃 비율
VOCAB_SIZE = 1200   # 단어 집합 크기 (예시)

model = Transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT
)


In [64]:
loss_function = nn.CrossEntropyLoss(ignore_index=sp.pad_id())

In [65]:
def get_lr_lambda(d_model, warmup_steps=4000):
    """
    Transformer 학습 시 사용되는 학습률 스케줄 함수(Step-wise learning rate scheduler) 생성.

    d_model: 모델 내부 표현 차원 (embedding 차원)
    warmup_steps: 학습률 warm-up 단계 수

    반환값: 현재 step에 따른 학습률 비율 계산 함수
    """
    d_model = float(d_model)

    def lr_lambda(step):
        # step은 0부터 시작하므로 +1로 보정
        step = step + 1

        # 수식: d_model^-0.5 * min(step^-0.5, step * warmup_steps^-1.5)
        # - 초기 단계(warmup)에서는 step * warmup_steps^-1.5가 작아 학습률 점점 증가
        # - warmup 이후에는 step^-0.5가 작아져 학습률 점점 감소
        return (d_model ** -0.5) * min(step ** -0.5, step * (warmup_steps ** -1.5))

    return lr_lambda


In [66]:
# Optimizer 정의
optimizer = optim.Adam(model.parameters(), betas=(0.9, 0.98), eps=1e-9)

# Scheduler 정의
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=get_lr_lambda(D_MODEL, warmup_steps=4000))


In [67]:
def accuracy_function(y_pred, y_true, pad_id=0):
    """
    모델 예측과 실제 정답을 비교하여 정확도를 계산합니다.
    
    Args:
        y_pred (Tensor): 모델 출력, shape = (batch_size, seq_len, vocab_size)
        y_true (Tensor): 실제 정답 토큰, shape = (batch_size, seq_len)
        pad_id (int): 패딩 토큰 ID (정답에 포함되어도 정확도 계산에서 제외)
    
    Returns:
        acc (Tensor): 배치 단위 정확도 (0~1)
    """
    # 1) vocab 차원에서 가장 높은 확률을 가진 토큰 선택
    preds = y_pred.argmax(dim=-1)  # shape = (batch_size, seq_len)

    # 2) 패딩이 아닌 위치만 선택
    mask = (y_true != pad_id)      # shape = (batch_size, seq_len)

    # 3) 예측이 정답과 같고, 패딩이 아닌 위치만 True
    correct = (preds == y_true) & mask

    # 4) 정확도 = 맞춘 토큰 수 / 패딩 제외 전체 토큰 수
    acc = correct.float().sum() / mask.float().sum()

    return acc

In [68]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

In [69]:
def train_step(model, batch, optimizer, loss_function, device):
    """
    한 배치(batch)에 대한 학습 단계 수행
    """
    model.train()  
    enc_input, dec_input, target = [x.to(device) for x in batch] 

    optimizer.zero_grad()  # 이전 gradient 초기화

    # 1) 모델 forward pass: (batch_size, seq_len, vocab_size) 출력
    logits = model(enc_input, dec_input)

    # 2) Loss 계산
    # CrossEntropyLoss는 (batch_size, vocab_size, seq_len) 형태 필요
    # pad_id 위치는 loss 계산에서 무시 가능
    loss = loss_function(logits.permute(0, 2, 1), target)

    # 3) Backpropagation
    loss.backward()
    optimizer.step()

    # 4) 정확도 계산 (패딩 토큰 제외)
    acc = accuracy_function(logits, target, pad_id=sp.pad_id())

    return loss.item(), acc

def train(model, dataloader, optimizer, loss_function, scheduler, num_epochs, device):
    """
    전체 학습 루프
    """
    model.to(device)

    for epoch in range(num_epochs):
        total_loss, total_acc = 0, 0

        for step, batch in enumerate(dataloader):
            # 배치 학습 수행
            loss, acc = train_step(model, batch, optimizer, loss_function, device)
            total_loss += loss
            total_acc += acc

            # 100 스텝마다 로그 출력
            if step % 100 == 0:
                print(f"[Epoch {epoch+1}, Step {step}] Loss: {loss:.4f}, Acc: {acc:.4f}")

            # 학습률 스케줄러 업데이트 (step 단위)
            scheduler.step()

        # 에포크 평균 Loss, Accuracy 출력
        avg_loss = total_loss / len(dataloader)
        avg_acc = total_acc / len(dataloader)
        print(f"Epoch {epoch+1} Completed - Avg Loss: {avg_loss:.4f}, Avg Acc: {avg_acc:.4f}")

In [70]:
%%time

train(
    model=model,
    dataloader=dataloader,
    optimizer=optimizer,
    loss_function=loss_function,
    scheduler=scheduler,
    num_epochs=50,  # 원하는 에폭 수
    device=device
) 

[Epoch 1, Step 0] Loss: 7.3226, Acc: 0.0000
[Epoch 1, Step 100] Loss: 7.3438, Acc: 0.0000
[Epoch 1, Step 200] Loss: 7.2717, Acc: 0.0000
[Epoch 1, Step 300] Loss: 7.2984, Acc: 0.0000
Epoch 1 Completed - Avg Loss: 7.3194, Avg Acc: 0.0006
[Epoch 2, Step 0] Loss: 7.2797, Acc: 0.0000
[Epoch 2, Step 100] Loss: 7.2320, Acc: 0.0025
[Epoch 2, Step 200] Loss: 7.1560, Acc: 0.0000
[Epoch 2, Step 300] Loss: 7.1279, Acc: 0.0000
Epoch 2 Completed - Avg Loss: 7.1743, Avg Acc: 0.0007
[Epoch 3, Step 0] Loss: 7.0592, Acc: 0.0000
[Epoch 3, Step 100] Loss: 6.9617, Acc: 0.0045
[Epoch 3, Step 200] Loss: 6.8840, Acc: 0.0054
[Epoch 3, Step 300] Loss: 6.8219, Acc: 0.0501
Epoch 3 Completed - Avg Loss: 6.8947, Avg Acc: 0.0261
[Epoch 4, Step 0] Loss: 6.7224, Acc: 0.0956
[Epoch 4, Step 100] Loss: 6.6300, Acc: 0.1469
[Epoch 4, Step 200] Loss: 6.4844, Acc: 0.1537
[Epoch 4, Step 300] Loss: 6.4286, Acc: 0.1637
Epoch 4 Completed - Avg Loss: 6.5178, Avg Acc: 0.1471
[Epoch 5, Step 0] Loss: 6.3549, Acc: 0.1622
[Epoch 5, St

# Step 5. 모델 평가하기

## Step 5-1. Greedy Search - 단순 가장 확률이 높은 단어 출력

In [71]:
def decoder_inference(model, sentence, sp, device='cpu', max_length=33):
    """
    한국어 SentencePiece 기반 Transformer 디코더 추론 (inference)

    Args:
        model: 학습된 Transformer 모델
        sentence (str): 입력 문장 (자연어 텍스트)
        sp: SentencePiece tokenizer
        device (str): 'cpu' 또는 'cuda'
        max_length (int): 생성할 최대 토큰 길이

    Returns:
        decoded_text (str): 모델이 생성한 최종 문장
        output_ids (list[int]): 생성된 토큰 ID 시퀀스
    """

    # 특수 토큰 ID 불러오기 (없을 경우 기본값 지정)
    START_TOKEN = sp.bos_id() if sp.bos_id() >= 0 else 1
    END_TOKEN   = sp.eos_id() if sp.eos_id() >= 0 else 2
    PAD_TOKEN   = sp.pad_id() if sp.pad_id() >= 0 else 0

    # 1. 입력 문장 전처리
    sentence = sentence.strip()

    # 2. 인코더 입력 구성
    enc_input_ids = [START_TOKEN] + sp.encode(sentence, out_type=int) + [END_TOKEN]
    enc_input = torch.tensor([enc_input_ids], dtype=torch.long, device=device)

    # 3. 디코더 입력 초기화 (BOS만 포함)
    dec_input = torch.tensor([[START_TOKEN]], dtype=torch.long, device=device)

    model.eval()
    with torch.no_grad():
        for _ in range(max_length):
            # 4. Forward pass
            logits = model(enc_input, dec_input)  # (1, seq_len, vocab_size)

            # 5. 마지막 시점의 예측 결과만 사용
            next_token_logits = logits[:, -1, :]  # (1, vocab_size)
            predicted_id = torch.argmax(next_token_logits, dim=-1).item()

            # 6. 종료 토큰이면 멈춤
            if predicted_id == END_TOKEN:
                break

            # 7. 예측된 토큰을 디코더 입력에 추가
            next_token = torch.tensor([[predicted_id]], device=device)
            dec_input = torch.cat([dec_input, next_token], dim=1)

    # 8. 최종 출력 시퀀스 정리
    output_ids = dec_input.squeeze(0).tolist()

    # 9. SentencePiece 디코딩 (BOS/EOS 제외)
    if END_TOKEN in output_ids:
        end_idx = output_ids.index(END_TOKEN)
        output_ids = output_ids[1:end_idx]  # [BOS] ... [EOS]
    else:
        output_ids = output_ids[1:]  # EOS 없으면 끝까지

    decoded_text = sp.decode(output_ids)

    return decoded_text, output_ids


In [72]:
def sentence_generation(model, sentence, sp, device='cpu'):
    """
    학습된 Transformer 모델로 한국어 입력 문장에 대한 응답 문장 생성

    Args:
        model: 학습된 Transformer 모델
        sentence (str): 입력 문장
        sp: SentencePiece tokenizer
        device (str): 'cpu' 또는 'cuda'

    Returns:
        predicted_sentence (str): 모델이 생성한 문장
    """
    # 디코더 인퍼런스를 통해 예측된 문장 및 토큰 ID 시퀀스 획득
    predicted_sentence, output_ids = decoder_inference(
        model=model,
        sentence=sentence,
        sp=sp,
        device=device
    )

    # 결과 출력
    print(f"입력: {sentence}")
    print(f"출력: {predicted_sentence}")
    print(f"토큰 ID 시퀀스: {output_ids}")

    return predicted_sentence


In [73]:
text = "훔쳐보는 것도 눈치 보임."
response = sentence_generation(model, text, sp, device=device)

입력: 훔쳐보는 것도 눈치 보임.
출력: 가 가 가 가 가 가 가 가 가 가 가 가 가 가 가 가 
토큰 ID 시퀀스: [4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4]


In [74]:
text = "짝사랑만큼 고통스러운 건 없겠지."
response = sentence_generation(model, text, sp, device=device)

입력: 짝사랑만큼 고통스러운 건 없겠지.
출력: 가 가 가 가 가 가 가 가 가 가 가 가 가 가 가 가 
토큰 ID 시퀀스: [4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4]


## Step 5-2. Beam Search - 후보 단어들을 유지하며 탐색

In [75]:
def decoder_inference_beam(model, sentence, sp, device='cpu', max_length=33, beam_width=5):
    START_TOKEN = sp.bos_id()
    END_TOKEN = sp.eos_id()

    sentence = preprocess_sentence(sentence)
    enc_input_ids = [START_TOKEN] + sp.encode(sentence) + [END_TOKEN]
    enc_input = torch.tensor([enc_input_ids], dtype=torch.long, device=device)

    sequences = [[START_TOKEN]]
    scores = [0.0]

    model.eval()
    with torch.no_grad():
        for _ in range(max_length):
            all_candidates = []

            for seq, score in zip(sequences, scores):
                # 이미 종료된 시퀀스는 그대로 유지
                if seq[-1] == END_TOKEN:
                    all_candidates.append((score, seq))
                    continue

                dec_input = torch.tensor([seq], dtype=torch.long, device=device)
                logits = model(enc_input, dec_input)
                next_logits = logits[:, -1, :]
                next_log_probs = torch.log_softmax(next_logits, dim=-1)

                topk_log_probs, topk_ids = torch.topk(next_log_probs, beam_width, dim=-1)

                for k in range(beam_width):
                    next_token = int(topk_ids[0, k].item())
                    candidate_seq = seq + [next_token]
                    candidate_score = score + float(topk_log_probs[0, k].item())
                    all_candidates.append((candidate_score, candidate_seq))

            # 점수 기준 상위 후보 유지
            all_candidates.sort(key=lambda x: x[0], reverse=True)
            all_candidates = all_candidates[:beam_width]

            sequences = [candidate[1] for candidate in all_candidates]
            scores = [candidate[0] for candidate in all_candidates]

            # 모든 후보 종료 시 반복 종료
            if all(seq[-1] == END_TOKEN for seq in sequences):
                break

    best_sequence = sequences[0]
    if best_sequence[-1] == END_TOKEN:
        best_sequence = best_sequence[:-1]

    return best_sequence


In [76]:
def sentence_generation_beam(model, sentence, sp, device='cpu'):
    output_ids = decoder_inference_beam(model, sentence, sp, device=device)
    predicted_sentence = sp.decode(output_ids)

    print(f"입력: {sentence}")
    print(f"출력: {predicted_sentence}")
    print(f"토큰 ID 시퀀스: {output_ids}")

    return predicted_sentence


In [77]:
text = "짝사랑만큼 고통스러운 건 없겠지."
response = sentence_generation_beam(model, text, sp, device=device)

입력: 짝사랑만큼 고통스러운 건 없겠지.
출력: 가 가 가 가 가요 .
토큰 ID 시퀀스: [1, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 8, 5]


In [78]:
text = "훔쳐보는 것도 눈치 보임."
response = sentence_generation_beam(model, text, sp, device=device)

입력: 훔쳐보는 것도 눈치 보임.
출력: 가 가 가 가 가 가 가요 .
토큰 ID 시퀀스: [1, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 4, 7, 8, 5]


In [79]:
text = "12시 땡"
response = sentence_generation_beam(model, text, sp, device=device)

입력: 12시 땡
출력: 가 가 가 가요 .
토큰 ID 시퀀스: [1, 4, 7, 4, 7, 4, 7, 4, 7, 8, 5]
